# ETL Processing Helper
 *An ETL project helper that loads data of US Accidents, road rankings, holiday events into a Postgres database in pieces.*
 



In [1]:
import pandas as pd
import requests

### Runs external CREATE statements and ETL

In [2]:
%run create_database.py
%run etl.py

ETL starting...
ETL Finished in 256.49986958503723 secs


### Load `accidents` data
* First viewing one row for the accidents table
* Look at the weather_timestamp variable
* Perform one row test run for the accidents table

In [8]:
df = pd.read_csv('./data/us_accidents.csv', index_col='ID', nrows=1)
#print(df)
print(df.info)

<bound method DataFrame.info of        Source    TMC  Severity           Start_Time             End_Time  \
ID                                                                         
A-1  MapQuest  201.0         3  2016-02-08 05:46:00  2016-02-08 11:00:00   

     Start_Lat  Start_Lng  End_Lat  End_Lng  Distance(mi)  ... Roundabout  \
ID                                                         ...              
A-1  39.865147 -84.058723      NaN      NaN          0.01  ...      False   

     Station   Stop Traffic_Calming Traffic_Signal Turning_Loop  \
ID                                                                
A-1    False  False           False          False        False   

    Sunrise_Sunset  Civil_Twilight Nautical_Twilight Astronomical_Twilight  
ID                                                                          
A-1          Night           Night             Night                 Night  

[1 rows x 48 columns]>


**Looking at the format of the Weather_Timestamp**

In [7]:
df2 = pd.read_csv('./data/us_accidents.csv', nrows=5)
for idx, r in df2.iterrows():
    print(r['Weather_Timestamp'])

2016-02-08 05:58:00
2016-02-08 05:51:00
2016-02-08 06:56:00
2016-02-08 07:38:00
2016-02-08 07:53:00


### Load `accidents` data

In [25]:
from create_database import connect_database
from sql_queries import *
import time

start_time = time.time()
conn, cur = connect_database()

df = pd.read_csv('./data/us_accidents.csv', nrows=1000)
for index, row in df.iterrows():
    
    # Handle null Weather_Timestamp
    weather_ts = pd.to_datetime(row.Weather_Timestamp)
    if pd.isnull(weather_ts):
        weather_ts = None      
    
    accident_row = (row.ID, 
                row.Severity, 
                row.Start_Time, 
                row.End_Time, 
                row.Start_Lat, 
                row.Start_Lng, 
                row.Description, 
                row.City, 
                row.County, 
                row.State,
                weather_ts, 
                row['Temperature(F)'], 
                row['Wind_Chill(F)'], 
                row['Humidity(%)'], 
                row['Pressure(in)'],
                row['Visibility(mi)'], 
                row['Wind_Speed(mph)'],
                row['Precipitation(in)'],
                row.Weather_Condition, 
                row.Amenity, 
                row.Bump, 
                row.Crossing,
                row.Give_Way, 
                row.Junction, 
                row.No_Exit, 
                row.Railway, 
                row.Roundabout, 
                row.Stop, 
                row.Traffic_Calming, 
                row.Traffic_Signal, 
                row.Turning_Loop,
                row.Sunrise_Sunset,
                row.Civil_Twilight,
                row.Nautical_Twilight,
                row.Astronomical_Twilight)
    
    cur.execute(insert_accidents, accident_row)   
    
conn.close()

time_delta = str(time.time() - start_time)
print('ETL Finished in {} secs'.format(time_delta) )

ETL Finished in 1.9375932216644287 secs


In [19]:
# Version 2 of loading the accidents table
# Only use subset of original csv

import time
from create_database import connect_database
from sql_queries import *

start_time = time.time()
conn, cur = connect_database()

df = pd.read_csv('./data/us_accidents.csv', nrows=1000)
ndf = df[['ID', 'Severity','Start_Time', 'End_Time', 'Start_Lat', 
         'Start_Lng', 'Description','City', 'County', 'State', 'Weather_Timestamp', 
           'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 
           'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',
            'Precipitation(in)', 'Weather_Condition', 'Amenity', 
            'Bump', 'Crossing', 'Give_Way', 'Junction', 
            'No_Exit', 'Railway', 'Roundabout', 'Stop', 
            'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop','Sunrise_Sunset',
            'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']]

tmp_file = 'file.csv'
ndf.to_csv(tmp_file, index=False, header=False)

file = open(tmp_file, 'r')
cur.copy_from(file, "accidents", sep=",", null="")
conn.close()

time_delta = str(time.time() - start_time)
print('ETL Finished in {} secs'.format(time_delta) )

ETL Finished in 0.10784792900085449 secs


### Load `time` data
* Loading one row for the time table

In [17]:
from create_database import connect_database
from sql_queries import *
import datetime
conn, cur = connect_database()

df = pd.read_csv('./data/us_accidents.csv', nrows=1)
for index, row in df.iterrows():
    dt = pd.to_datetime(row.Start_Time)
    time_row = (dt,datetime.date(dt.year, dt.month, dt.day), dt.hour,dt.day, dt.month, dt.year, dt.weekday())
    print("Time row: ", time_row)
    print(type(time_row[0]))
    cur.execute(insert_time, time_row)
conn.close()
    

Time row:  (Timestamp('2016-02-08 05:46:00'), datetime.date(2016, 2, 8), 5, 8, 2, 2016, 0)
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


### Load `holidays` data

* View the holidays data

In [14]:
holidays_df = pd.read_csv('./data/usholidays.csv')
print(holidays_df.columns)

holidays_df.head()

Index(['Unnamed: 0', 'Date', 'Holiday'], dtype='object')


,Unnamed: 0,Date,Holiday
0,0,2010-12-31,New Year's Day
1,1,2011-01-17,"Birthday of Martin Luther King, Jr."
2,2,2011-02-21,Washington's Birthday
3,3,2011-05-30,Memorial Day
4,4,2011-07-04,Independence Day


### Load `roadRankings` data

* Perform one `INSERT` into the roadRankings Table 

In [4]:
from create_database import connect_database
from sql_queries import *

conn, cur = connect_database()

ranking_df = pd.read_csv('./data/state_road_ratings.csv',delimiter=',',nrows=3)
ranking_df.columns = ['overall_rank','year','state','commute_time', 'public_transit','road_quality','bridge_quality']
for index, row in ranking_df.iterrows():
    row_rank = (row.overall_rank, row.year, row.state, row.commute_time, row.public_transit, row.road_quality, row.bridge_quality)
    cur.execute(insert_road_rank, row_rank)
conn.close()

### Load `accidentsAnalysis` data


In [4]:
import datetime
from create_database import connect_database
from sql_queries import *
conn, cur = connect_database()

# read the acciedents csv
df = pd.read_csv('./data/us_accidents.csv', nrows=1)

def retrieve_row_match(cur, select_statement, match):
    """ retrieves first column of the select statement, 
        otherwise, returns None
    """
    cur.execute(select_statement, match)
    result = cur.fetchone()
    return result[0] if result else None
    
for index, row in df.iterrows():
    
    # retrieve the matching holiday id if exists 
    dt = pd.to_datetime(row.Start_Time)
    dt_date = (datetime.date(dt.year, dt.month, dt.day),)
    holiday_id = retrieve_row_match(cur, select_holiday, dt_date)

    # retrieve year and state to match the road ranking id
    ranking_match = (dt.year, row.State)
    ranking_id = retrieve_row_match(cur, select_rank, ranking_match)
    
    analysis_row = (row.ID, dt, row.State, holiday_id, ranking_id)
    cur.execute(insert_analysis, analysis_row)
conn.close()        